In [1]:
import os
from mysql import connector
from dotenv import load_dotenv
import pandas as pd

# Access environment variables
load_dotenv()

NAME = os.getenv("DB_NAME")
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
IP = os.getenv("DB_IP")
PORT = os.getenv("DB_PORT")

class Database:
    _connection = None  # 클래스 변수로 연결 상태 관리

    @staticmethod
    def get_connection():
        if Database._connection is None or not Database._connection.is_connected():
            Database._connection = connector.connect(
                database = NAME,
                user = USER,
                password = PASSWORD,
                host = IP,
                port = PORT
            )
        return Database._connection

try:
    conn = Database.get_connection()
    cursor = conn.cursor()
    
    cursor.execute("""SELECT 
                        book.ID,
                        registration_year,
                        registration_month,
                        get_course,
                        DDC,
                        publication_year,
                        location,
                        duration,
                        COALESCE(SUM(`2004`) + SUM(`2005`) + SUM(`2006`) + SUM(`2007`) + SUM(`2008`) +
                                 SUM(`2009`) + SUM(`2010`) + SUM(`2011`) + SUM(`2012`) + SUM(`2013`) +
                                 SUM(`2014`) + SUM(`2015`) + SUM(`2016`) + SUM(`2017`) + SUM(`2018`) +
                                 SUM(`2019`) + SUM(`2020`) + SUM(`2021`) + SUM(`2022`) + SUM(`2023`) +
                                 SUM(`2024`), 0) AS total_rent,
                        COALESCE(rent_count, 0) AS rent_count,
                        COALESCE(book_count, 0) AS book_count
                    FROM 
                        book
                    LEFT JOIN 
                        recent_rent ON book.ID = recent_rent.ID
                    LEFT JOIN 
                        rent_count ON book.ID = rent_count.ID
                    LEFT JOIN 
                        ISBN_rent_count ON book.ISBN = ISBN_rent_count.ISBN
                    GROUP BY 
                        book.ID, registration_year, registration_month, get_course, DDC, publication_year, location, duration
                    ORDER BY 
                        book.ID
                    """)

    rows = cursor.fetchall()
    
    cursor.close()
except connector.Error as e:
    print(f"Error: {e}")

df = pd.DataFrame(rows, columns=['ID', '등록연도', '등록월', '수서방법', '분류코드', '출판연도', '소장위치', '최근대출', '총 대출 횟수', 'rent_count', 'book_count'])

In [2]:
df[52478:52500]

,ID,등록연도,등록월,수서방법,분류코드,출판연도,소장위치,최근대출,총 대출 횟수,rent_count,book_count
52478,SS_052649,2013,6,이용자희망,155.7,2013,4층인문,3298.0,1,2,2
52479,SS_052650,2013,6,이용자희망,153.35,2013,4층인문,3524.0,3,3,1
52480,SS_052651,2013,6,이용자희망,741.6,2013,4층인문,1422.0,4,6,2
52481,SS_052652,2013,6,이용자희망,741.6,2013,4층인문,2186.0,2,6,2
52482,SS_052653,2013,6,이용자희망,746.43,2013,4층인문,NaN,0,1,1
52483,SS_052654,2013,6,이용자희망,746.43,2013,4층인문,2807.0,1,1,1
52484,SS_052655,2013,6,이용자희망,780.15,2013,4층인문,3116.0,2,5,2
52485,SS_052656,2013,6,이용자희망,808.5,2013,4층인문,3419.0,2,2,1
52486,SS_052657,2013,6,이용자희망,808.5,2013,4층인문,NaN,0,2,1
52487,SS_052658,2013,6,이용자희망,780.15,2013,보존서고,3124.0,3,5,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          299315 non-null  object 
 1   등록연도        299315 non-null  int64  
 2   등록월         299315 non-null  int64  
 3   수서방법        299315 non-null  object 
 4   분류코드        299315 non-null  object 
 5   출판연도        299315 non-null  object 
 6   소장위치        299315 non-null  object 
 7   최근대출        160686 non-null  float64
 8   총 대출 횟수     299315 non-null  object 
 9   rent_count  299315 non-null  object 
 10  book_count  299315 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 25.1+ MB


In [4]:
# 1. ID 처리: '_' 기준으로 분리, 뒤쪽 값만 사용 후 int로 변환
df['ID'] = df['ID'].str.split('_').str[-1].astype(int)

# 2. 수서 방법을 category로 변환
df['수서방법'] = df['수서방법'].astype('category')

# 3. 분류코드를 float로 변환
df['분류코드'] = df['분류코드'].astype(float)

# 4. 출판연도를 int로 변환
df['출판연도'] = df['출판연도'].astype(int)

# 5. 소장위치를 category로 변환
df['소장위치'] = df['소장위치'].astype('category')

# 6. 최근대출을 int로 변환하고 NaN 값을 7305로 설정
df['최근대출'] = df['최근대출'].fillna(7305).astype(int)

# 7. 총 대출 횟수, rent_count, book_count를 int로 변환
df['총 대출 횟수'] = df['총 대출 횟수'].astype(int)
df['rent_count'] = df['rent_count'].astype(int)
df['book_count'] = df['book_count'].astype(int)

# 결과 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ID          299315 non-null  int64   
 1   등록연도        299315 non-null  int64   
 2   등록월         299315 non-null  int64   
 3   수서방법        299315 non-null  category
 4   분류코드        299315 non-null  float64 
 5   출판연도        299315 non-null  int64   
 6   소장위치        299315 non-null  category
 7   최근대출        299315 non-null  int64   
 8   총 대출 횟수     299315 non-null  int64   
 9   rent_count  299315 non-null  int64   
 10  book_count  299315 non-null  int64   
dtypes: category(2), float64(1), int64(8)
memory usage: 21.1 MB


In [5]:
df_onehot = pd.get_dummies(df)

In [6]:
df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   rent_count   299315 non-null  int64  
 8   book_count   299315 non-null  int64  
 9   수서방법_기타      299315 non-null  bool   
 10  수서방법_사서선정    299315 non-null  bool   
 11  수서방법_수서정보없음  299315 non-null  bool   
 12  수서방법_수업지정    299315 non-null  bool   
 13  수서방법_이용자희망   299315 non-null  bool   
 14  수서방법_학과신청    299315 non-null  bool   
 15  소장위치_4층인문    299315 non-null  bool   
 16  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(8)
memory usage: 22.8 MB


In [7]:
y_data = df_onehot['rent_count'] / df_onehot['book_count']

y_data = y_data.fillna(0)
y_data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 299315 entries, 0 to 299314
Series name: None
Non-Null Count   Dtype  
--------------   -----  
299315 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB


In [8]:
df_onehot.drop(columns=['rent_count', 'book_count'], inplace=True)

df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


In [9]:
# 모듈 불러오기
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [10]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df_onehot, y_data, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(209520, 15)
(89795, 15)
(209520,)
(89795,)


In [11]:
# 모델 생성
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,   # 트리 개수
    learning_rate=0.1,  # 학습률
    max_depth=3,        # 트리 최대 깊이
    random_state=42
)

In [12]:
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [13]:
y_pred = xgb_reg.predict(X_test)

In [14]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

RMSE: 2.4716
R^2 : 0.8450


In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.8, 1.0]
}

xgb_reg = xgb.XGBRegressor(random_state=42)
grid_cv = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # 회귀의 경우, MSE 계열 사용
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_cv.fit(X_train, y_train)
print("최적 파라미터:", grid_cv.best_params_)
print("최고 성능 (MSE):", -grid_cv.best_score_)

# 최적 모델을 이용해 테스트 세트에서 R^2 점수 확인
best_estimator = grid_cv.best_estimator_
y_pred_test = best_estimator.predict(X_test)

r2_test = r2_score(y_test, y_pred_test)
print("테스트 세트 R^2 :", r2_test)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
최적 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.8}
최고 성능 (MSE): 5.632087801713857
테스트 세트 R^2 : 0.8677535075976153


In [17]:
# 모델 생성
xgb_reg = xgb.XGBRegressor(
    n_estimators = 500,   # 트리 개수
    learning_rate = 0.1,  # 학습률
    max_depth = 7,        # 트리 최대 깊이
    subsample = 0.8,
    random_state = 42
)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

RMSE: 2.2828
R^2 : 0.8678
[CV] END learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=   3.8s
[CV] END learning_rate=0.1, max_depth=3, n_estimators=400, subsample=0.8; total time=  14.0s
[CV] END learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=   5.8s
[CV] END learning_rate=0.1, max_depth=5, n_estimators=300, subsample=0.8; total time=  16.0s
[CV] END learning_rate=0.1, max_depth=7, n_estimators=100, subsample=0.8; total time=   8.7s
[CV] END learning_rate=0.1, max_depth=7, n_estimators=200, subsample=0.8; total time=  15.4s
[CV] END learning_rate=0.1, max_depth=7, n_estimators=400, subsample=1.0; total time=  29.9s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=300, subsample=1.0; total time=   9.9s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=500, subsample=1.0; total time=  16.0s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=400, subsample=0.8; total time=  23.9s
[CV] END learning_rate=0.01, max_depth=7,